## Option 1: Auto-refresh cache for a set of Looks
First, some boring setup stuff. We import the API and create a connection client.

In [1]:
import lookCache
import swagger_client as looker
client = lookCache.apiConnect('config.yml')

This next bit creates the looker objects we'll use to do all the neat stuff.

In [2]:
look = looker.LookApi(client)
space = looker.SpaceApi(client)

Now we generate a list of all spaces. We can use a list comprehension to easily filter to what we want, in this case all public spaces. Note we have to cast the results to integers.

In [3]:
allSpace = space.all_spaces()
targetSpaces = [int(i.id) for i in allSpace if i.id != 'lookml' and i.is_personal == False]

In [4]:
print(targetSpaces)

[1, 2, 10, 11, 12, 18, 19, 20, 28, 29, 30, 31, 42, 51, 54, 56]


Now we can loop through this list of spaces, extract all the looks contained therein...

In [5]:
targetLooks = []
for s in targetSpaces:
    spaceLook = space.space_looks(s)
    spaceLookId = [i.id for i in spaceLook]
    targetLooks.append(spaceLookId)


...and flatten it into a single list. If this gets really large you may want to yield a generator instead of a list.

In [6]:
flattened = [item for sublist in targetLooks for item in sublist]
print(flattened)

[25, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 98, 99, 26, 27, 28, 29, 100, 72, 73, 74, 76, 77, 79, 81, 56, 57, 58, 59, 61, 63, 86, 87, 88, 89, 90, 91, 92, 93, 95, 97, 118, 119, 120, 121, 122, 123, 124, 125, 136, 138, 139, 112, 113, 114, 115, 116, 117, 104, 105, 106, 107, 108, 109, 110, 126, 127, 128, 129, 130, 131, 132, 133, 140, 142, 143, 144, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 173, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 174]


Now we could loop through this list of looks and run each one with cache set to False. In the interest of time we'll only do one...

In [7]:
look.run_look(look_id = 173, result_format='json', cache = False)

"[{'order_items.returned': 'No', 'order_items.count': 38013}, {'order_items.returned': 'Yes', 'order_items.count': 758}]"

## Option 2: Utilize dashboard pre-fetching
We can also utilize Looker's dashboard prefetch ability. If you have a heavily used dashboard with a set number of common combinations of filters this can be a solid option. But it does require some setup. First, we need to create a dashboard object.

In [8]:
dash = looker.DashboardApi(client)

By default, dashboards are not setup for prefetching. We need to update our dashboard's configuration. You only need to run this once per dashboard.

In [ ]:
dash.update_dashboard(dashboard_id=9, body={"load_configuration":"prefetch_cache_run"})

Now we can create a couple of prefetches. After we run this we'll switch over to the dashboard to see it in action

In [9]:
dash.create_dashboard_prefetch(dashboard_id=9, 
                               body={"ttl": 600,
                                     "dashboard_filters": [
                                        {
                                        "name": "Date",
                                        "value":"3 months ago for 3 months",
                                        "can": {}
                                        }
                                     ]
                                    }
                              )

dash.create_dashboard_prefetch(dashboard_id=9, 
                               body={"ttl": 600,
                                     "dashboard_filters": [
                                        {
                                        "name": "Date",
                                        "value":"5 months ago for 5 months",
                                        "can": {}
                                        }
                                     ]
                                    }
                              )

{'access_filters': [],
 'can': {'create': True},
 'computation_time': 0.4179999828338623,
 'created_at': datetime.datetime(2017, 5, 3, 13, 9, 43, 769000, tzinfo=tzutc()),
 'dashboard_filters': [{'can': None,
                        'name': 'Date',
                        'value': '5 months ago for 5 months'}],
 'hit_count': 0,
 'result_size_bytes': 6898,
 'touched_at': datetime.datetime(2017, 5, 3, 13, 9, 45, 381000, tzinfo=tzutc()),
 'ttl': 600}

Let's check on the status of our '3 month' prefetch. Note the 'hit-count' metric... we'll come back to this in a moment.

In [10]:
dash.dashboard_prefetch(dashboard_id = 9, 
                        dashboard_filters = '[{ "name": "Date", "value":"3 months ago for 3 months", "can": {} }]'
                       )

{'can': {'create': True},
 'computation_time': 0.28299999237060547,
 'created_at': datetime.datetime(2017, 5, 3, 13, 9, 42, 63000, tzinfo=tzutc()),
 'hit_count': 1,
 'result_size_bytes': 6031,
 'touched_at': datetime.datetime(2017, 5, 3, 13, 9, 53, 123000, tzinfo=tzutc()),
 'ttl': 600,
 'url': None,
 'value': {'36': "{'data': [{'users.zipcode': {'value': '95605'}, "
                 "'orders.count': {'value': 8}}, {'users.zipcode': {'value': "
                 "'90068'}, 'orders.count': {'value': 7}}, {'users.zipcode': "
                 "{'value': '92018'}, 'orders.count': {'value': 7}}, "
                 "{'users.zipcode': {'value': '90019'}, 'orders.count': "
                 "{'value': 7}}, {'users.zipcode': {'value': '92123'}, "
                 "'orders.count': {'value': 6}}, {'users.zipcode': {'value': "
                 "'94166'}, 'orders.count': {'value': 5}}, {'users.zipcode': "
                 "{'value': '93590'}, 'orders.count': {'value': 5}}, "
                 "{'users.

Hope that inspires some hacking. Have fun!